In [0]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StringType,IntegerType,DateType,StructField,StructType,FloatType
from pyspark.sql.functions import max,min,mean,format_number,count,corr,year,month

In [0]:
SS = SparkSession.builder.appName('excercise').getOrCreate()
#Start Session

In [0]:
df = SS.read.table('walmart_stock_csv')
df1 = SS.sql("select * from walmart_stock_csv")

#Load dataframe

In [0]:
df.columns
#load columns

Out[15]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [0]:
df.printSchema()

#Schema does not look proper

root
-- Date: string (nullable = true)
-- Open: string (nullable = true)
-- High: string (nullable = true)
-- Low: string (nullable = true)
-- Close: string (nullable = true)
-- Volume: string (nullable = true)
-- Adj Close: string (nullable = true)

In [0]:
df = df.withColumn('Date',df['Date'].cast(DateType()))
df = df.withColumn('Open',df['Open'].cast(FloatType()))
df = df.withColumn('High',df['High'].cast(FloatType()))
df = df.withColumn('Low',df['Low'].cast(FloatType()))
df = df.withColumn('Close',df['Close'].cast(FloatType()))
df = df.withColumn('Volume',df['Volume'].cast(IntegerType()))
df = df.withColumn('Adj Close',df['Adj Close'].cast(FloatType()))

#Updating schema1

In [0]:
df.printSchema()

root
-- Date: date (nullable = true)
-- Open: float (nullable = true)
-- High: float (nullable = true)
-- Low: float (nullable = true)
-- Close: float (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: float (nullable = true)

In [0]:
new_schema = StructType([StructField('Date',DateType(),True),StructField('Open',FloatType(),True),
                         StructField('High',FloatType(),True),StructField('Low',FloatType(),True),
                         StructField('Close',FloatType(),True),StructField('Volume',IntegerType(),True),
                         StructField('Adj Close',FloatType(),True)])
df2 = SS.read.csv('/FileStore/tables/walmart_stock-1.csv',schema=new_schema)
#Updating schema2

In [0]:
df2.printSchema()

root
-- Date: date (nullable = true)
-- Open: float (nullable = true)
-- High: float (nullable = true)
-- Low: float (nullable = true)
-- Close: float (nullable = true)
-- Volume: integer (nullable = true)
-- Adj Close: float (nullable = true)

In [0]:
df.collect()[0:5]
#First 5 rows

Out[42]: [Row(Date=datetime.date(2012, 1, 3), Open=59.970001220703125, High=61.060001373291016, Low=59.869998931884766, Close=60.33000183105469, Volume=12668800, Adj Close=52.61923599243164),
 Row(Date=datetime.date(2012, 1, 4), Open=60.209999084472656, High=60.349998474121094, Low=59.470001220703125, Close=59.709999084472656, Volume=9593300, Adj Close=52.07847595214844),
 Row(Date=datetime.date(2012, 1, 5), Open=59.349998474121094, High=59.619998931884766, Low=58.369998931884766, Close=59.41999816894531, Volume=12768200, Adj Close=51.825538635253906),
 Row(Date=datetime.date(2012, 1, 6), Open=59.41999816894531, High=59.45000076293945, Low=58.869998931884766, Close=59.0, Volume=8069400, Adj Close=51.45922088623047),
 Row(Date=datetime.date(2012, 1, 9), Open=59.029998779296875, High=59.54999923706055, Low=58.91999816894531, Close=59.18000030517578, Volume=6679300, Adj Close=51.616214752197266)]

In [0]:
df.head(5)
#First 5 rows

Out[124]: [Row(Date=datetime.date(2012, 1, 3), Open=59.970001220703125, High=61.060001373291016, Low=59.869998931884766, Close=60.33000183105469, Volume=12668800, Adj Close=52.61923599243164),
 Row(Date=datetime.date(2012, 1, 4), Open=60.209999084472656, High=60.349998474121094, Low=59.470001220703125, Close=59.709999084472656, Volume=9593300, Adj Close=52.07847595214844),
 Row(Date=datetime.date(2012, 1, 5), Open=59.349998474121094, High=59.619998931884766, Low=58.369998931884766, Close=59.41999816894531, Volume=12768200, Adj Close=51.825538635253906),
 Row(Date=datetime.date(2012, 1, 6), Open=59.41999816894531, High=59.45000076293945, Low=58.869998931884766, Close=59.0, Volume=8069400, Adj Close=51.45922088623047),
 Row(Date=datetime.date(2012, 1, 9), Open=59.029998779296875, High=59.54999923706055, Low=58.91999816894531, Close=59.18000030517578, Volume=6679300, Adj Close=51.616214752197266)]

In [0]:
df1 = df.describe()
df1.show()
#Describe to get 5 point summary

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
summary| Open| High| Low| Close| Volume| Adj Close|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
 count| 1258| 1258| 1258| 1258| 1258| 1258|
 mean|72.35785375452572| 72.8393880756178|71.91860094964979|72.38844997363553|8222093.481717011|67.23883840200064|
 stddev|6.768090251767697|6.768186825250206|6.744075739203606|6.756859160119612| 4519780.8431556|6.722609385249684|
 min| 56.39| 57.06| 56.3| 56.42| 2094900| 50.36369|
 max| 90.8| 90.97| 89.25| 90.47| 80898100| 84.914215|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+

In [0]:
df1.select(df1['summary'],
           format_number(df1['Open'].cast(FloatType()),2).alias('Open'),
           format_number(df1['High'].cast(FloatType()),2).alias('High'),
           format_number(df1['Low'].cast(FloatType()),2).alias('Low'),
           format_number(df1['Close'].cast(FloatType()),2).alias('Close'),
           df1['Volume'].cast(IntegerType())).alias('Volume').show()

+-------+--------+--------+--------+--------+--------+
summary| Open| High| Low| Close| Volume|
+-------+--------+--------+--------+--------+--------+
 count|1,258.00|1,258.00|1,258.00|1,258.00| 1258|
 mean| 72.36| 72.84| 71.92| 72.39| 8222093|
 stddev| 6.77| 6.77| 6.74| 6.76| 4519780|
 min| 56.39| 57.06| 56.30| 56.42| 2094900|
 max| 90.80| 90.97| 89.25| 90.47|80898100|
+-------+--------+--------+--------+--------+--------+

In [0]:
df.withColumn('HV Ratio',df['High']/df['Volume']).select('HV Ratio').show()

+--------------------+
 HV Ratio|
+--------------------+
4.819714682786927E-6|
6.290848662516662E-6|
 4.66941298944916E-6|
 7.36733843444859E-6|
8.915604814435727E-6|
8.644477449144044E-6|
9.351828386844425E-6|
 8.29141562102703E-6|
7.712212051589609E-6|
7.071764777688419...|
1.015495462653464...|
 6.5763540967921E-6|
 5.90145296180676E-6|
8.547679390846264E-6|
8.420709512685392E-6|
1.041448335142357...|
8.316075435382035E-6|
9.721183804158345E-6|
8.029435987746889E-6|
6.307432228123159E-6|
+--------------------+
only showing top 20 rows

In [0]:
df.filter(df['High'] == df.select(max('High')).collect()[0][0]).select(['Date','High']).show()

+----------+-----+
 Date| High|
+----------+-----+
2015-01-13|90.97|
+----------+-----+

In [0]:
df.agg(format_number(mean('close'),2).alias('Average')).show()

+-------+
Average|
+-------+
 72.39|
+-------+

In [0]:
df.select([max(df['Volume']),min(df['Volume'])]).show()

+-----------+-----------+
max(Volume)|min(Volume)|
+-----------+-----------+
 80898100| 2094900|
+-----------+-----------+

In [0]:
df.filter('Close<60').select(count('Date')).collect()[0][0]

Out[119]: 81

In [0]:
df.filter('High>80').select(count('High')).collect()[0][0]/df.select(count('High')).collect()[0][0]*100

Out[81]: 9.141494435612083

In [0]:
df.select(corr(df['High'],df['Volume'])).show()

+-------------------+
 corr(High, Volume)|
+-------------------+
-0.3384326061799556|
+-------------------+

In [0]:
df.withColumn('Year',year('Date')).select(['Year','High']).groupBy('Year').max().select(['Year','max(High)']).show()

+----+---------+
Year|max(High)|
+----+---------+
2015| 90.97|
2013| 81.37|
2014| 88.09|
2012| 77.6|
2016| 75.19|
+----+---------+

In [0]:
df.withColumn('Month',month('Date')).select(['Month','Close']).groupBy('Month').mean().select(['Month','avg(Close)']).sort('Month').show()

+-----+-----------------+
Month| avg(Close)|
+-----+-----------------+
 1| 71.4480196131338|
 2|71.30680438169499|
 3|71.77794376266337|
 4|72.97361900692894|
 5|72.30971685445533|
 6| 72.4953774506191|
 7|74.43971944078106|
 8| 73.0298185521906|
 9|72.18411782208611|
 10| 71.5785454489968|
 11|72.11108927207418|
 12|72.84792482628012|
+-----+-----------------+